In [23]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2
scikit-learn-intelex==2021.20220215.212715


In [17]:
!python -V

Python 3.9.12


In [1]:
import pickle
import pandas as pd
import numpy as np 

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [6]:
std_y_pred = np.std(y_pred)

print("Standard Deviation of y_pred:", std_y_pred)

Standard Deviation of y_pred: 6.247488852238703


In [13]:
# Define year and month variables
year = 2023
month = 3  # March

# Assuming df is your DataFrame containing trip records
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [16]:
df_result = pd.DataFrame()
df_result['ride_id']=df['ride_id']
df_result['prediction_duration']= y_pred

In [20]:
!mkdir output_file 

In [21]:
df_result.to_parquet(
    'output_file/yellow_tripdata_2023-03.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

Size of the output file: 65.46201992034912 bytes (65.46 MB)
